InternVL
====

**InternVL: Scaling up Vision Foundation Models and Aligning for Generic Visual-Linguistic Tasks**

 * Paper: https://arxiv.org/pdf/2312.14238

![InterVL Overview](../assets/internvl_overview.png)
![InternVL Training](../assets/internvl_training.png)
![InternVL Usages](../assets/internvl_usages.png)

```bash
git clone https://github.com/OpenGVLab/InternVL.git InternVL_repo

conda create -n internvl python=3.9 -y
conda activate internvl

cd InternVL_repo
pip install -r requirements.txt
pip install -r requirements/clip_benchmark.txt
```

In [ ]:
from PIL import Image
import textwrap
import torch
from transformers import AutoTokenizer, AutoModel
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "OpenGVLab/InternVL3-1B"

model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device
).eval()

tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True, use_fast=False
)

/home/pyml/anaconda3/envs/internvl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pyml/anaconda3/envs/internvl/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FlashAttention2 is not installed.


/home/pyml/anaconda3/envs/internvl/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Helper Functions

In [2]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

### Single-image inference

In [ ]:
# set the max number of tiles in `max_num`
pixel_values = load_image(
    '../samples/plants.jpg', max_num=12
).to(torch.bfloat16).to(device)
generation_config = dict(max_new_tokens=1024, do_sample=True)

# single-image single-round conversation
question = '<image>\nPlease describe the image shortly.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(textwrap.fill(f'User: {question}\nAssistant: {response}', 80))

# single-image multi-round conversation
question = '<image>\nHow many leaves the large plant on the right vase has?'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(textwrap.fill(f'User: {question}\nAssistant: {response}', 80))


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image> Please describe the image shortly. Assistant: The image features
two potted plants, one placed on a black pot with a blue rim and the other in a
shiny gold container. Both plants are set against a wooden floor against a plain
wall.
User: <image> How many leaves the large plant on the right vase has? Assistant:
The large plant on the right vase has 8-10 leaves.
